# The Monty Hall Problem

R.C. Alvarado

This notebook implements a solution to the Monty Hall problem as given in the paper, ["What's So Hard about the Monty Hall Problem" (Alvarado 2024)](./alvaado-2024.pdf). 

# Model

The problem is represented as a probabilistic graphic model (PGM) with six random variables represented as nodes.

![](monty-hall-as-pgm.png)

Each of the nodes in the graph correspond to a random variable. These are represented here:

$D = \{A, B, C\}$

$S(): s \in \{keep, switch, flip\}$

$X(): x \in D$

$G_1(): g_1 \in D$

$H(x, g_1): h \in D - \{x \cup g_1\}$

$G_2(g_1, h, s): \left\{ \begin{array}{ll}
        s = switch: & g_2 \in D - \{h \cup g_1\} \\
        s = keep: & g_2 = g_1 \\
        s = flip: & g_2 \in D - h
\end{array} \right.\\$

$R(x, g_2): r = x \equiv g_2$

# Class

The following class defines methods for each of the random variables described above.

It also defines methods for playing the game in order to test hypotheses about the probabilities\
 of winning and losing based on the strategy $S$ to switch or keep doors after the host opens a door.

We model the game as a state frame with each door $\in D$ as a row and each random variable as a column.

Each column may take values $\{0,1\}$. For example, here is a game in which the contestant wins by switching:

| $D$ | $X$ | $G_1$ | $H$ | $G_2$ | $R$ |
|-----|---|---|---|---|---|
| $A$ | 1 | 0 | 0 | 1 | 1 |
| $B$ | 0 | 1 | 0 | 0 | 0 |
| $C$ | 0 | 0 | 1 | 0 | 0 |


$S$ is not modeled as a column since it is a meta-variable that corresponds to the hypothesis about which strategy to choose.

Note that in this form the dataframe acts as a truth table, and each function can be expressed as truth functions, like so:

$H(x, g_1): h \in \neg (x \lor g_1)$

$G_2(g_1, h, s=switch): g_2 \in \neg (g_1 \lor h)$ 

$R(x, g_2): r = x \land g_2$

Each state table assumes a value of $S$.

In [57]:
import pandas as pd

class MontyHall():
    
    # Define primitive sets
    S  = "keep switch flip".split() # The three strategies
    D = "A B C".split() # The three doors

    # Initialize the game 
    def __init__(self, s = 'switch'):
        if s not in self.S:
            raise ValueError(f"s must be one of {S}.")
        self.s = s

        # We represent the state of game as a dataframe
        self.game = pd.DataFrame(dict(X = None), index=pd.Index(self.D, name='D'))
            
    # Hide the car
    def X(self):
        self.game.X = 0
        self.game.loc[self.game.sample().index, 'X'] = 1
        
    # Contestant's first guess
    def G1(self):
        self.game['G1'] = 0
        self.game.loc[self.game.sample().index, 'G1'] = 1
        
    # Host's reveal
    def H(self):
        self.game['H'] = 0
        # idx = (self.game.X + self.game.G1) == 0
        self.game.loc[self.game[(self.game.X == 0) & (self.game.G1 == 0)].sample().index, 'H'] = 1
        
    # Contestant's second guess
    def G2(self):
        self.game['G2'] = 0
        if self.s == 'switch':
            # idx = (self.game.G1 + self.game.H) == 0
            self.game.loc[self.game[(self.game.G1 == 0) & (self.game.H == 0)].index, 'G2'] = 1
        elif self.s == 'keep':
            self.game.G2 = self.game.G1
        elif self.s == 'flip':
            # idx = self.game.H == 0
            self.game.loc[self.game[self.game.H == 0].sample().index, 'G2'] = 1
        else:
            raise ValueError(f"Strategy not one of {self.S}.")
        
    # Result of the game
    def R(self):
        self.game['R'] = self.game.X * self.game.G2
        self.result = self.game.R.sum()
        
    # Play a single game
    def play_one(self):
        self.X()
        self.G1()
        self.H()
        self.G2()
        self.R() 
        return self
    
    # Play many games to test hypotheses
    def play_many(self, n=1000):
        return sum([self.play_one().result for i in range(n)]) / n    

# Test

We test the model with the three strategies.

In [58]:
mh = MontyHall()

In [59]:
print("s\tp")
for s in MontyHall.S:
    mh.s = s
    r = mh.play_many()
    print(s, r, sep="\t")

s	p
keep	0.303
switch	0.642
flip	0.485


Example of single game.

In [65]:
mh.s = 'switch'
mh.play_one()
print(mh.game)
print('Strategy:', mh.s)
print('Result:', mh.result)

   X  G1  H  G2  R
D                 
A  1   0  0   1  1
B  0   0  1   0  0
C  0   1  0   0  0
Strategy: switch
Result: 1
